Overlapping Domain Decomposition Methods
===
Let $\{ \Omega_i \}$ be a non-overlapping decomposition of $\Omega$ into open sub-domains, i.e.

$$
\Omega_i \cap \Omega_j = 0 \quad \text{ for } i \neq j
$$

and

$$
\overline \Omega = \bigcup \ \overline \Omega_i
$$

Let $H_i = \operatorname{diam} \Omega_i$. To simplify notation we assume that all $H_i$ are in the same order of magnitude, and write $H$.


In addition to $\Omega_i$, we define enlarged sub-domains

$$
\widetilde \Omega_i = {U}_{cH} (\Omega_i) = \{ x \in \Omega : \operatorname{dist} (x,\Omega_i) < c H \},
$$

where the constant $c$ is order of one.


The overlapping domain decomposition method is a sub-space decomposition of the global finite element space $V_h \subset H^1_{0,D}$ into local spaces supported on the overlapping sub-domains $\widetilde \Omega_i$, i.e.

$$
V_i = \{ v \in V_h : v = 0 \text{ on } \Omega \setminus \widetilde \Omega_i \}
$$

The overlapping domain decomposition method is an additive Schwarz method with the space splitting

$$
V_h = \sum V_i
$$

We are solving Dirichlet problems on the overlapping sub-domains, and adding up the solutions.

Experiments with overlapping DD
---

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

We start from a coarse grid, where we define the overlapping sub-domains as vertex patches: All elements connected with vertex i form the $i^{th}$ sub-domain. To visualize it, we define an $L_2$ finite
element function of order 0:

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))
fesdom = L2(mesh, order=0, autoupdate=True)
fes = H1(mesh, order=1, dirichlet="bottom", autoupdate=True)
gfdom = GridFunction(fesdom, multidim=mesh.nv, nested=True, autoupdate=True)
for el in mesh.Elements(VOL):
    for v in el.vertices:
        gfdom.vecs[v.nr][el.nr] = 1
        
for l in range(5):
    mesh.Refine()

Draw (gfdom)

Setup the problem. The sub-spaces are defined via BitArrays with 
cleared bits for dofs not belonging to elements not in $\widetilde \Omega_i$:

In [ ]:
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()

pre = None
for v in gfdom.vecs:
    mask = BitArray(fes.FreeDofs())
    for el in fes.Elements(VOL):
        if v[el.nr] == 0:
            for d in el.dofs:
                mask[d] = False
    # print (mask)
    invi = a.mat.Inverse(freedofs=mask, inverse="sparsecholesky")
    pre = pre+invi if pre else invi     

In [ ]:
from ngsolve.la import EigenValues_Preconditioner
lam = list(EigenValues_Preconditioner(a.mat, pre))
print ("lammin, lammax=", lam[0], lam[-1])
print ("kappa=", lam[-1]/lam[0])

We observe the following behaviour:
* the condition number is independent of the refinement level
* the condition number grows with the number of sub-domains

Analysis of the DD preconditioner
---

Theorem: The eigenvalues of the domain decomposition preconditioner are bounded as

$$
\sigma(C_{DD}^{-1} A) \subset [c_1 H^2, c_2 ]
$$

*Outline of the proof*: We apply the Additive Schwarz Lemma. The ASM lemma provides the representation

$$
u^T C_{DD} u = \inf_{u = \sum u_i \atop  u_i \in V_i} \sum_i \| u_i \|_A^2
$$

By assuming regularity of the domains, only a small, fixed number of sub-domains overlap. From that we get the upper bound $c_2$ for the spectrum. For the lower bound $c_1 H^2$ we have to define explicitly a decomposition of a finite element function $u$ into sub-space finite element functions. This requires some technical tools.

**Partition on unity (PUM):**

We call the set of functions $\{ \psi_i \}$ a partition of unity for the domain decomposition $\widetilde \Omega_i$ iff

$$
\text{support} \{ \psi_i \}  \subset \widetilde \Omega_i
$$

$$
0 \leq \psi_i \leq 1
$$

and

$$
\sum \psi_i = 1
$$

For the definition of the overlapping domain decomposition we can define a PUM as follows:

$$
\psi_i^0(x) = \max \{0,  1 - \frac{1}{cH_i} \operatorname{dist} \{ x, \Omega_i \} \}
$$

These functions are $1$ inside $\Omega_i$, $0$ outside $\widetilde \Omega_i$, and decay with a maximal derivative 

$$
| \nabla \psi_i^0 |_{\infty}  \leq \frac{1}{c H_i}
$$

However, these function do not sum up to 1, so we scale
$$
\psi_i(x) := \frac{\psi_i^0(x)} {\sum \psi_i^0(x) }
$$

**$H^1$-stable quasi-interpolation operators:**

We need local interpolation-like operators
$$
\pi_h : H^1 \rightarrow V_h
$$

which are projectors onto $V_h$, i.e. $I_h v_h = v_h$ for $v_h \in V_h$, are continuous in the $H^1$ semi-norm

$$
\| \nabla \pi_h v \|_{L_2(\Omega)} \prec \| \nabla v \|_{L_2(\Omega)},
$$

and satisfy the approximation estimate
$$
 \| \tfrac{1}{h} (v - \pi_h v) \|_{L_2(\Omega)} \prec \| \nabla v \|_{L_2(\Omega)}.
$$

We cannot use a Clément-type quasi-interpolation operator: Such an operator defines nodal values by some local averaging.

We have now the tools to conduct the proof for the sub-space decomposition:

For a given $u \in V_h$ we define

$$
u_i = \pi_h (\psi_i u)
$$

These $u_i$ are in $V_i$, i.e. are localized finite element functions.

By linearity of the quasi-interpolation, the partition of unity,
and the projection property they are a decomposition of the given function $u$:

$$
\sum u_i = \sum \pi_h (\psi_i u) = \pi_h \big( \sum \psi_i u  \big)
= \pi_h u = u
$$


Finally, we have to bound the norm of the decomposition. First,
we use the boundedness of the quasi-interpolation in the $H^1$ semi-norm:

$$
\| \nabla u_i \|_{L_2} = \| \nabla (\pi_h \psi_i u) \|_{L_2}
\prec \| \nabla (\psi_i u) \|_{L_2} 
$$

We continue with the product rule, and bounds for the pum-functions and their derivatives:

\begin{eqnarray*}
\| \nabla u_i \|_{L_2} & \prec & \| (\nabla \psi_i) u + \|
\end{eqnarray*}

Overlapping DD Methods with coarse grid
---